Learning from this source: https://realpython.com/python-send-email/#sending-fancy-emails

This code scrapes recipes from Bon Appetit URLs, and sends them to a specified email address.

Emails have to come from a gmail account, and you'll have to go to [settings --> "Allow less secure apps" --> ON ] in order to execute. 

In [242]:
import requests
import smtplib
import numpy as np
import pandas as pd
import ssl
import email

from pandas import Series,DataFrame
from numpy.random import randn
from bs4 import BeautifulSoup
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [243]:
def PrintRecipe(url):
    '''
    DOCSTRING: A function to print out the ingredients and steps for any recipe from Bon Appetit.
    INPUTS: Bon Appetit recipe url.
    OUTPUTS: None.
    '''
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # Pull the HTML for recipe name, ingredients list, and steps. 
    recipe_title = soup.find_all('h1',class_='post__header__hed')
    ingredients = soup.find_all('li',class_='ingredient')
    steps = soup.find_all('li',class_="step")
    
    # Extract the clean text for the recipe name. 
    clean_title = []
    for title in recipe_title:
        clean_title.append(title.text.strip())
    
    # Extract the clean text for the ingredients.
    ingredients_list = []
    for ingredient in ingredients:
        ingredients_list.append(ingredient.text.strip())

    # Extract the clean text for the recipe instructions. 
    steps_list = []
    for step in steps:
        steps_list.append(step.text.strip())
    
    # Here we print the ingredients and steps.
    print('\033[1m' + 'Recipe: \n\n' + '\033[0m' + clean_title[0] + '\n\n--------\n')
    print('\033[1m' + "Ingredients:\n")
    for ingredient in ingredients_list:
        print('\033[0m' + ingredient)
    print('\n--------'+'\033[1m' + '\n\nInstructions:')
    i = 1
    for step in steps_list:
        print('\033[0m')
        print(f"Step {i}: \n{step}")
        i+=1
    print('\n--------\n\nBon appetit!')
    
    return

In [244]:
def GetRecipe(url):
    '''
    DOCSTRING: A function to scrape the ingredients and steps for any recipe from Bon Appetit, and return it as a dictionary with keys for Title and Instructions.
    INPUTS: Bon Appetit recipe url.
    OUTPUTS: Title and instructions inside of 'recipe' dictionary.
    '''
    
    #f = open('recipe.txt','w')
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # Pull the HTML for recipe name, ingredients list, and steps. 
    recipe_title = soup.find_all('h1',class_='post__header__hed')
    ingredients = soup.find_all('li',class_='ingredient')
    steps = soup.find_all('li',class_="step")
    
    # Extract the clean text for the recipe name. 
    clean_title = []
    for title in recipe_title:
        clean_title.append(title.text.strip())
    
    # Extract the clean text for the ingredients.
    ingredients_list = []
    for ingredient in ingredients:
        for char in ingredient: 
            if char == '½':
                char = '.5'
        ingredients_list.append(ingredient.text.strip())

    # Extract the clean text for the recipe instructions. 
    steps_list = []
    for step in steps:
        steps_list.append(step.text.strip())
        
    # Message will hold the body text of the email.
    message = ''
    
    # Here we print the ingredients and steps.
    #message = message + '\033[1m' + ('Recipe: \n\n' + '\033[0m' + clean_title[0] +'\n\n--------\n\n')
    message = message + "Ingredients:\n\n"
    for ingredient in ingredients_list:
        message = message + ingredient + '\n'
    message = message + ('\n--------' + '\n\nInstructions:\n\n')
    i = 1
    for step in steps_list:
        message = message + (f"Step {i}:" + f"\n{step}\n\n")
        i+=1
    message = message + ('--------\n\n' + 'Bon appetit!')

    recipe_title = clean_title[0]
    
    recipe = {}
    recipe['Title'] = recipe_title 
    recipe['Instructions'] = '\n' + '--------\n\n' + message
    
    return(recipe)


In [245]:
def BonAppetit(url,me,password,you):
    
    '''
    DOCSTRING: This function takes a recipe, email logins, and destination address, and sends the recipe to that inbox as an attachment.
    INPUTS: URL for a Bon Appetit recipe, email login, email password, destination email address.
    OUTPUTS: None
    '''
    
    # Retrieve the recipe as a dictionary, where the first key is the title and the second is the instructions.
    recipe = GetRecipe(url)
    
    title = recipe['Title']
    instructions = recipe['Instructions']
    
    # Email layout:
    subject = f"A recipe from Bon Appetit: {title}"
    body = f"See below for today's recipe, {title}." + instructions
    
    sender_email = me
    receiver_email = you
    password = password
    
    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    
    # Add body to email
    message.attach(MIMEText(body, "plain"))
    
    # Convert message to string
    text = message.as_string()
    
    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)
    
    return

In [ ]:
me = input("From which email address should the message come? ")
password = input("Enter the password for sending email account: ")
you = input("Who are you sending this to? ")

url = input("What recipe would you like to send? ")
recipe = BonAppetit(url,me,password,you)